# Perturbation Map Analysis of Binding Free Energies of Lysozyme

This notebook will guide you through how to obtain binding free energies from the using network tools. In particular we want to compute relative binding free energies with respect to benzene as a ligand to Lysozyme. 

The notebook forms part of the CCPBio-Sim workshop **Alchemical Free Energy Simulation Analysis with analyse_freenrg** run on the 11th of April 2018 at the University of Bristol.

*Author: Antonia Mey   
Email: antonia.mey@ed.ac.uk*

**Reading time of the document: 20 mins**

## Let's start with the necessary imports

In [ ]:
%pylab inline
import seaborn as sbn
sbn.set_style("ticks")
sbn.set_context("notebook", font_scale = 2)
from networkanalysis.networkanalysis import *
from networkanalysis.experiments import *
from networkanalysis.plotting import *
from networkanalysis.stats import *

### Perturbation maps, quick reminder
In the previous notebook we looked at computing all the relative free energies between ligands for which we had setup simulations according to this perturbation map:   
![foo](images/perturbation_map.png)

But, what we are of course really interested in is the relative free energy differences of the eight ligands we have simulated: benzene, indole, indene, benzofuran, p-xylene, o-xylene, isoputylbenzene and n-butylbenzne. The following notebook will guide you through exactly this and use the fact that the perturbation map is a graph or network in mathematical terms and in this way utilise a lot of network algorithms to facilitate the analysis. 

## Experimental data
We need to read experimental data in order to be able to compare computed and experimental free energies. For this purpose a free energy file with experimental data taken from this [paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2104542/pdf/nihms28543.pdf) has conveniently be provided. 

In [ ]:
#Initialise the experimental data
exp_data = ExperimentalData()

In [ ]:
filename = 'data/FE_exp.csv'
exp_data.read_free_energies(filename, kcal=True)

The experimental data is read into a dictionary which we can access in the following way:

In [ ]:
exp_data.freeEnergiesInKcal

In this case here errors are arbitrarily chosen rather than using the ones provided from the paper. 

## Simulation data 
Let's read in the simulation data, which we had computed in Exercise02. For convenience a version of the free energy file has been placed in the data directory. 

In order to read in the data we need to initialise a `perturbationGraph`. This object is an extension to the NetworkX library which allows easy computation of network properties, such as weighted paths etc, which is what we need in order to compute our free energies. 

In [ ]:
pG = PerturbationGraph()
pG.populate_pert_graph('data/FE.csv')

We can now compute weighted paths with respect to a particular ligand/compound. For the purpose of this exercise we chose benzene, as it is the compound that forms the substructure of all other simulated compounds. 

In [ ]:
pG.compute_weighted_avg_paths('benzene')

We can format free energies in a convenient way that will remove intermediates or average over multiple binding modes. This is not necessary here as we did not use multiple binding modes in the simulation or had intermediates. We can call the function none the less. 

In [ ]:
pG.format_free_energies(weighted=True)

`pG.write_free_energies` allows to write the free energies in a nicely formatted way to a file or just display to the standard output. 

In [ ]:
pG.write_free_energies(pG.freeEnergyInKcal)

Computing cycle closure is a good way of ensuring that your free energies are well converged. Usually cycle closure errors of less than 1 kcal/mol are a good indication of a reliable dataset. Have a look at the cycles below that result in a poor cycle closure error. (i.e. a threshold of larger than 1 kcal/mol.)

In [ ]:
pG.get_cycles()

**Task: What does the above suggest for the dataset?**

### Plotting results
freenrgworkflows has a couple of common plotting functions, namely a scatter and bar plot to compare experimental and computed free energy values. We initialise the plotter by giving it an experimental set of free energy values and computed set of free energy values. 

In [ ]:
fep = FreeEnergyPlotter(exp_data.freeEnergiesInKcal,pG.freeEnergyInKcal)

The bar plot is called in the following way:

In [ ]:
fep.plot_bar_plot()

The same data in a scatter plot looks like this:

In [ ]:
fep.plot_scatter_plot()

### Statistics
Now we might want to know something about the statistics of the computed free energies, i.e what is the correlation between the two datasets and what is the mean unsigned error in kcal/mol. The statistics module has basic functionality for that and computes samples of the dataset in order to look at the spread and reliability of the correlation estimate. 


In [ ]:
stats = freeEnergyStats()

The next cell will generate 10000 samples of the dataset in order to assess the spread in the measures used here: correlation coefficient R, Kendall tau and mean unsigned error. 

In [ ]:
stats.generate_statistics(pG.freeEnergyInKcal,exp_data.freeEnergiesInKcal,repeats=10000)

In [ ]:
print ('Summary of statistics:')
print ('----------------------')
print ('Correlation coefficient: %.2f ± %.2f' %(stats.R, stats.R_std))
print ('R^2:                     %.2f ± %.2f' %(stats.R2, stats.R2_std))
print ('Kendall tau:             %.2f ± %.2f' %(stats.tau, stats.tau_std))
print ('mean unsigned error:     %.2f ± %.2f [kcal/mol]' %(stats.mue, stats.mue_std))

**Tasks:**   
- Look up exactly what R, R^2 and tau measure and make sure you understand what the values mean. 
- You will find a second dataset (data/FE_second.csv) as well as the TI data (data/FE_ti.csv) in the data directory. Run the same analysis with the second data set. What do you observe? 

In [ ]:
# Read in second set of simulation data


In [ ]:
# Look at plots of second simulation set with respect to the experimental values



In [ ]:
# Run the statistical analysis of the second data set with respect to the experimental values. 



## A little quiz for the end
Please have a quick look at this quiz to see what you have learned from this notebook: https://goo.gl/forms/Onsdxftz1DnLDekM2

Congratulations you are now able to analyse alchemical free energy calculations!